# 0_01 : New try: slicing filtered texts manually

1. Vou filtrar as sessoes em que se falam as nossas palavras chaves
2. Depois eu vou apenas nesses arquivos e ajusto eles manualmente
3. No fim eu volto e re-filtro, mandendo apenas os Tagesordnung que valem a pena ler

In [1]:
import numpy as np
import pandas as pd
import re
import glob

### Primeira parte: procurar quais sao as sessoes que têm algum dos termos:

# #Abrindo DF com todas as sessoes
general_df_original = pd.read_csv('0_general_df.csv', sep=';', encoding='UTF-8')

general_df = general_df_original.drop('Unnamed: 0', axis=1)
general_df = general_df.drop_duplicates()

# Define the search terms and forbidden terms
search_terms_holz = [r'\bholz\w*', r'\bforst\w*', r'\bwald\w*', r'\bbioökonomie', r'\bbioenergie', r'\bmöbel\w*']

forbidden_terms = ['Holzner', 'Holzleitner', 'Waldneukirchen', 'Holzmann', 'Waldegg', 'Waldheims', 'Waldorf', 'Waldvier', 'Waldei',
                   'Holzkreuze', 'Holzweg', 'Wald4tler', 'Walddorf', 
                  'Waldkirchen', 'Waldburg', 'Waldorfschule', 'Waldspaziergang', 'Holzfuß', 'Waldzell', 'Holzleit',
                  'Waldbe', 'Waldneukirchen', 'Holzgau', 'waldorfmathematisch', 'Waldwürfe', 'Waldviertelbahn', 'Waldheim', 'Walding', 'Waldenstein', 
                  'Waldviertlerin', 'Holzlärm', 'Holzchild', 'Waldviertelautobahn', 'Holzinger', 'Waldviertler', 'Waldbrand', 'Waldbrände', 'Land- und forstwirtschaftliche Landeslehrpersonen-Dienstrechtsgesetz',
                  'Land- und forstwirtschaftliche Landesvertrags­lehr­personengesetz']

# Create the regular expression pattern with word boundaries
padrao_holz = r"\b(" + r"|".join(search_terms_holz) + r")\b"
regex_pattern = re.compile(padrao_holz)

dfs = []

#### Aqui comeca o loop:
for index, row in general_df.iterrows():
    print(index)
    text = row['text']

    #fazendo a lista dos matches:
    list_of_matches = re.findall(regex_pattern, text)

    #para cada lista dos matches, verificar se a palavra está na lista dos forbiden terms
    cleaned_matches = [x for x in list_of_matches if x not in forbidden_terms]
    
    
    #se cleaned matches é > 0, manter a linha no df novo, senao, ir para a próxima:
    if len(cleaned_matches) >= 1:

        print(cleaned_matches)
        termos = ', '.join(cleaned_matches)
        

        df_temp = pd.DataFrame([[termos]], columns=["terms_founded"])
        df_temp['id'] = row['id']
        df_temp['dokumentart'] = row['dokumentart']
        df_temp['typ'] = row['typ']
        df_temp['vorgangsbezug_anzahl'] = row['vorgangsbezug_anzahl']
        df_temp['dokumentnummer'] = row['dokumentnummer']
        df_temp['wahlperiode'] = row['wahlperiode']
        df_temp['herausgeber'] = row['herausgeber']
        df_temp['pdf_hash'] = row['pdf_hash']
        df_temp['datum'] = row['datum']
        df_temp['titel'] = row['titel']
        df_temp['fundstelle'] = row['fundstelle']
        df_temp['vorgangsbezug'] = row['vorgangsbezug']
        df_temp['aktualisiert'] = row['aktualisiert']
        df_temp['text'] = row['text']

        dfs.append(df_temp)

    else:
        next

# Concatenate the list of DataFrames into a single DataFrame
filtered_df = pd.concat(dfs)

# remove duplicates:
filtered_df = filtered_df.drop_duplicates(subset='pdf_hash')

# Create a new column from 1 to n
filtered_df['index'] = range(1, len(filtered_df) + 1)

# Set the new column as the index
filtered_df.set_index('index', inplace=True)

# Assign empty string ('') to 'file_name' column
filtered_df['section_file_name'] = ''

#Now I'll save TXT files of the whole section, so I can divide the sections into Tagesordnungpunkt manually and later bring it back to the dataframe

dir = "C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\New_German_Sten_Protokollen\\filtered_sections\\"

for index, row in filtered_df.iterrows():
    text = row['text']
    datum = row['datum']
    dokumentnummer = row['dokumentnummer']
    dokumentnummer = re.sub("/", '_', dokumentnummer)

    section_file_name = datum + '_' + dokumentnummer  + '.txt'      #Aqui o filename é apenas a data e o número da sessao
    filtered_df.at[index, 'section_file_name'] = section_file_name
    
    section_file_name = dir + datum + '_' + dokumentnummer  + '.txt'      #Aqui é pra salvar
    
    # Open the file in write mode
    # Salvando as sessoes
    with open(section_file_name, "w", encoding="utf-8") as file:
        file.write(text)

    
    print(section_file_name , " saved to the file.")

0
1
2
3
4
['forst']
5
6
7
['waldschutz']
8
9
['forstpolitische', 'forstlichen', 'waldbaulichen', 'forstwirtschaftlicher', 'forstwirtschaftliche']
10
11
12
13
14
15
16
17
18
['forstwirtschaftliche']
19
['waldreichen', 'forstwirtschaftlichen']
20
21
22
23
24
['holzt']
25
26
27
28
29
30
['holzschnittartig']
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
['holz']
56
57
58
59
60
61
62
63
64
65
66
['waldpolitische']
67
68
['holzen']
69
70
['forst']
71
72
73
74
['forstlichen', 'forstlichen', 'forstwirtschaftlichen']
75
76
77
78
79
['holzschnittartig']
80
81
82
['holzschnittartig']
83
84
['holzschnittartigen']
85
86
87
88
['forstwirtschaftlichen']
89
90
91
92
93
94
['forstlicher', 'forstlicher']
95
['forstwirtschaftlichen', 'forstwirtschaftlichen', 'waldbrandgefahr256']
96
97
98
99
100
['forstlicher', 'forstwirtschaftlichen', 'forstlicher']
101
['wald']
102
103
104
105
106
['forstliche', 'forstlichem', 'forstlichen', 'forstliche', 'forstlichem', 'forstlichen', 'wald

## Arquivos verificados

Como eu nao conseguia automaticamente detectar o Tagesordnungspunkt, eu salvei os arquivos e verifiquei um a um onde comecava e terminada cada TOP

```###TAGESORDNUNGSPUNKT###```

Arquivos revisados:
2019-02-13_19_79_
2019-04-10_19_94_
2019-05-10_19_99_
2019-09-10_19_110_
2019-09-25_19_114_
2019-09-26_19_115_
2019-09-27_19_116_
2019-10-18_19_119_
2019-11-26_19_129_
2019-12-13_19_135_
2019-12-18_19_136_
2019-12-19_19_137_
2020-01-29_19_142_
2020-05-07_19_158_
2020-07-01_19_169_
2020-09-17_19_176_
2020-09-30_19_179_
2020-10-30_19_187_
2020-11-05_19_189_
2020-12-16_19_201_
2021-01-13_19_203_
2021-01-27_19_205_
2021-01-28_19_206_
2021-02-24_19_211_
2021-03-05_19_216_
2021-04-15_19_221_
2021-04-21_19_223_
2021-04-22_19_224_
2021-05-19_19_229_
2021-05-20_19_230_
2021-06-23_19_235_
2021-06-24_19_236_
2021-11-18_20_3_
2021-12-10_20_7_
2021-12-16_20_9_
2022-01-14_20_12_
2022-02-17_20_17_
2022-03-17_20_21_
2022-03-22_20_23_
2022-03-24_20_25_
2022-05-18_20_36_
2022-10-14_20_61_
2022-11-11_20_67_
2022-11-30_20_72_
2022-12-01_20_73_
2023-01-26_20_82_
2023-02-08_20_84_
2023-03-01_20_87_

In [2]:
# Segunda parte: dividindo o texto completo da sessao por Tagesordnungpunkt

# Abrir cada um dos negócios 
# Adicionar no df como texto alternativo
# Depois recortar no ###TAGESORDNUNGSPUNKT###

dir = "C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\New_German_Sten_Protokollen\\filtered_sections_marked\\"

filtered_df['text_marked'] = ''

#Aqui eu estou abrindo os textos marcados:
for index, row in filtered_df.iterrows():
    section_file_name = row['section_file_name']
    section_file_name = dir + section_file_name 

    # Open the file in read mode
    with open(section_file_name, 'r', encoding='utf-8') as file:
        # Read the contents of the file
        contents = file.read()

    filtered_df.at[index, 'text_marked'] = contents



In [4]:
################################################################################

# Aqui estou dividindo os 'text_marked' , que sao os textos que eu alterei:

dfs = []

for index, row in filtered_df.iterrows():
    text = row['text_marked']

    # Dividing the text
    tops = text.split('###TAGESORDNUNGSPUNKT###')
    tops = tops[1:]

    # Nessa parte do loop, eu vou verificar quais sao os TOP que contém os termos de madeira que a gente estabeleceu lá em cima:

    for top in tops:
        # Finding matches
        list_of_matches = re.findall(regex_pattern, top)

        # Checking forbidden terms
        cleaned_matches = [x for x in list_of_matches if x not in forbidden_terms]

        if len(cleaned_matches) >= 1:
            print(cleaned_matches)
            termos = ', '.join(cleaned_matches)

            # Creating DataFrame
            df = pd.DataFrame([top], columns=['tops'])
            df['dokumentnummer'] = row['dokumentnummer']
            df['wahlperiode'] = row['wahlperiode']
            df['plenarprotokoll_id'] = row['id']
            df['pdf_hash'] = row['pdf_hash']
            df['datum'] = row['datum']
            df['titel'] = row['titel']
            df['wahlperiode'] = row['wahlperiode']
            df['section_file_name'] = row['section_file_name']        # Filename ainda é data + sessao:

            dfs.append(df)
        else:
            next

# O tops_df é um dataframe que contém as informacoes da sessao e o texto APENAS do TOP:
tops_df = pd.concat(dfs)

#Deu certo!

# Nessa terceira parte eu vou salvar os arquivos do TOP separadamente, pois assim eu consigo verificar a existência de algum padrao para dividir os reden:

#Criando Ids
tops_df['top_id'] = range(1, len(tops_df) + 1)
# Criando uma coluna vazia para o nome dos file name do TOP
tops_df['top_file_name'] = ''

#Diretório onde eu vou salvar os TOPS
dir = "C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\New_German_Sten_Protokollen\\filtered_tops\\"

tops_df.reset_index(drop=True, inplace=True)

for index, row in tops_df.iterrows():
    text = row['tops']
    datum = row['datum']
    dokumentnummer = row['dokumentnummer']
    dokumentnummer = re.sub("/", '_', dokumentnummer)
    top_id = row['top_id']
    
    top_file_name = datum + '_' + dokumentnummer + '_' + str(top_id) + '.txt'

    tops_df.at[index, 'top_file_name'] = top_file_name
    
    print(top_file_name, " saved to the file.")

    top_file_name = dir + top_file_name

    # Open the file in write mode
    with open(top_file_name, "w", encoding="utf-8") as file:
        file.write(text)




#### FOI AQUI QUE ALGO DEU ERRADO. FICARAM REGISTRADOS TODOS OS FILENAMNE COM O MESMO NOME

['waldschutz']
['forstpolitische', 'forstlichen', 'waldbaulichen', 'forstwirtschaftlicher', 'forstwirtschaftliche']
['forstwirtschaftliche']
['waldreichen']
['forstwirtschaftlichen']
['holzt']
['holzschnittartig']
['holz']
['waldpolitische']
['holzen']
['forst']
['forstlichen', 'forstlichen', 'forstwirtschaftlichen']
['holzschnittartig']
['holzschnittartig']
['holzschnittartigen']
['forstwirtschaftlichen']
['forstlicher']
['forstwirtschaftlichen']
['waldbrandgefahr256']
['forstwirtschaftlichen']
['forstlicher']
['wald']
['waldreichen']
['forstliche', 'forstlichem', 'forstlichen', 'waldreichsten', 'forstliche', 'forstlichem', 'forstlichen']
['forstliche', 'forstlichem', 'forstlichen']
['forstwirtschaftlichen', 'forstwirtschaftlichen']
['holzen']
['holzschnittartiger']
['wald', 'waldeigentuemer', 'holzbe', 'wald', 'wald', 'wald', 'waldzustandserhebung', 'forstwirtschaftlichen']
['forstwirtschaftlichen', 'forstwirtschaftlichen']
['forstwirtschaftlichen', 'forstwirtschaftlichen']
['forstli

Agora sao 56 documentos, mas ainda nao está bom porque sao muitas falas e certamente nao temos os termos em todas as falas. Entao eu vou fazer a mesma coisa para dividir as falas: Vou colocar o marcador ```###REDE###``` nas falas dos parlamentares, e depois re-dividir os TOP por rede.

In [5]:
# Sao duas divisoes: quando o presidente fala (nome do presidente:) e quando um parlamentar fala (nome do partido entre ():)

#primeiro, eu preciso saber todos os nomes dos Vice-presidentes:
diretorio = 'C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\New_German_Sten_Protokollen\\filtered_tops\\*.txt'  

arquivos_txt = glob.glob(diretorio)

presidentes = []
padrao = r"(Vizepräsident \w{3,}\s\w{3,})|(Vizepräsidentin \w{3,}\s\w{3,})"

for arquivo in arquivos_txt:
    with open(arquivo, 'r', encoding = "utf-8") as file:
        conteudo = file.read()
        
        resultado = re.findall(padrao, conteudo)
        presidentes = presidentes + resultado

presidentes_sem_duplicatas = list(set(presidentes))

# desnesting the list:
presidentes = []

for m in range(len(presidentes_sem_duplicatas)):
   for n in range (len(presidentes_sem_duplicatas[m])):
      presidentes.append(presidentes_sem_duplicatas[m][n])

presidentes = list(filter(None, presidentes))

# Agora eu vou adicionar ###REDE### antes da fala de cada uma das pessoas que vai falar:
df_TOP_rede = []

for arquivo in arquivos_txt:
    with open(arquivo, 'r', encoding = "utf-8") as file:
        conteudo = file.read()

        # # #buscando os presidentes
        for pres in presidentes:
            if re.findall(pres, conteudo):
                conteudo = re.sub(pres, "###REDE###" + pres, conteudo)

        # # Buscando os speeches:
        padrao = r"\w{2,}\s\w{2,}\s\(.*\):"
        matches = re.findall(padrao, conteudo)

        for match in matches:
            match_escaped = re.escape(match) 
            conteudo = re.sub(match_escaped, "###REDE###" + match , conteudo)

        df_temp = pd.DataFrame([[conteudo]], columns=['TOP'])
        a = re.sub(r"C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\New_German_Sten_Protokollen\\filtered_tops\\", "", arquivo)
        a = re.sub(r"\.txt", "", a)
        df_temp['file'] = a

        df_TOP_rede.append(df_temp)

df_final = pd.concat(df_TOP_rede, ignore_index=True)

# Eu vou usar o nome do file para fazer left join

In [6]:
# Agora é a hora de re-dividir os textos:

dfs = []

for index, row in df_final.iterrows():
    text = row['TOP']

    # Dividing the text
    reden = text.split('###REDE###')
    reden = reden[1:]

    for rede in reden:
        # Finding matches com o que tem Holz:
        list_of_matches = re.findall(regex_pattern, rede)

        # Checking forbidden terms
        cleaned_matches = [x for x in list_of_matches if x not in forbidden_terms]

        if len(cleaned_matches) >= 1:
            print(cleaned_matches)
            termos = ', '.join(cleaned_matches)

            # Creating DataFrame
            df = pd.DataFrame([rede], columns=['reden'])
            df['file_name'] = row['file']
            df['terms'] = termos

            dfs.append(df)
        else:
            next

# Concatenate the list of DataFrames into a single DataFrame
reden_df = pd.concat(dfs)

#Deu certo!

def get_parlamentar(r):
    """"
    Funcao que pega o nome do parlamentar a partir de um input de texto
    """
    parlamentar = re.findall(r'^.*\):', r )
    if parlamentar:
        parlamentar = re.sub(":", "", str(parlamentar[0]))
    else:
        for pres in presidentes:
            if re.findall(pres, r ):
                parlamentar = re.findall(pres, r)
                parlamentar = re.sub(":", "", str(parlamentar[0]))
    return parlamentar

# Aplicando a função em cada elemento da coluna 'A' usando apply()
reden_df['abgeordenete'] = reden_df['reden'].apply(lambda x: get_parlamentar(x))

# Deu certo <3

['holzen']
['forst', 'waldes']
['forstwirtschaftlicher']
['forstwirtschaftlichen']
['waldreichen']
['holzbasierten']
['waldbraende', 'waldwirtschaftabkommen']
['forstlichen']
['holzschnittartig']
['forstwirtschaftliche', 'forstwirtschaftliche', 'forstwirtschaftlich', 'forstwirtschaftlich']
['holzig']
['holzschnittartig']
['walderhaltende']
['forstlichen', 'forstlichen']
['waldreichsten']
['forstlichen']
['forsten']
['forstwirtschaftlichen']
['wald']
['forstlichen']
['forst']
['forst', 'forst']
['forstlicher']
['holzschnittartig']
['forstlichen', 'forstlichen']
['waldpolitische']
['forstwirtschaftliche']
['forstwirtschaftlichen', 'forstwirtschaftlichen']
['forstwirtschaftliches']
['forstlichem']
['waldverträgliches']
['forstwirtschaftliche']
['forstwirtschaftlichen', 'forstwirtschaftlichen']
['wald', 'waldeigentuemer', 'holzbe']
['wald']
['wald', 'wald', 'waldzustandserhebung']
['forstwirtschaftlichen']
['forstwirtschaftlichen', 'forstwirtschaftlichen']
['holzschnittartiger']
['holzen']

In [13]:
reden_df.head(1)

,reden,file_name,terms,abgeordenete
0,Peter Stein (Rostock) (CDU/CSU):\nHerr Präside...,2019-02-13_19_79_56,holzen,Peter Stein (Rostock) (CDU/CSU)


In [45]:
# Finalmente, chegou a hora de juntar as informacoes sobre a sessao parlamentar aos redes:

infos_df = tops_df
infos_df = infos_df.drop('tops', axis = 1)   # derrubando o texto do tops, pq eu nao preciso dele
infos_df = infos_df.rename(columns={'top_file_name': 'file_name'})

# define function to remove '.txt' extension from file names
def remove_txt_extension(file_name):
    return re.sub('\.txt', '', file_name)

# apply function to 'file_name' column
infos_df['file_name'] = infos_df['file_name'].apply(remove_txt_extension)

reden_df_final = pd.merge(reden_df, infos_df, how='left', on='file_name')

reden_df_final = reden_df_final.drop_duplicates()

reden_df_final['reden_id'] = range(1 , len(reden_df_final) + 1)

reden_df_final['reden_file_name'] = reden_df_final['abgeordenete'] + reden_df_final['file_name'] + '_' + reden_df_final['reden_id'].astype(str) + '.txt'


def fix_names(x):
    x = re.sub('\(' , '', x)
    x = re.sub('\)' , '', x)
    x = re.sub('\/' , '_', x)
    x = re.sub('\s' , '_', x)
    return x

reden_df_final['reden_file_name'] = reden_df_final['reden_file_name'].apply(lambda x: fix_names(x))

reden_df_final.head()
# Agora vou finalmente exportar isso!

#Diretório onde eu vou salvar os TOPS
dir = "C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\New_German_Sten_Protokollen\\filtered_reden\\"

for index, row in reden_df_final.iterrows():
    text = row['reden']
    file_name =  dir + row['reden_file_name']
    
    # Open the file in write mode
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(text)

In [46]:
reden_df_final.to_csv('reden_df_final.csv', encoding='utf-8', sep=';')